In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from nltk.classify.maxent import MaxentClassifier
from sklearn.model_selection import train_test_split
import nltk
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
import math
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
import seaborn as sns
import gc

from catboost import CatBoostClassifier

In [3]:
import warnings
warnings.filterwarnings("ignore")

In [4]:
from utils import Timer

In [5]:
df = pd.read_csv('../data/train_data_1.csv')


In [6]:
df_ = df[['phone_number','price_vnd']]

In [7]:
del df

In [8]:
df_

,phone_number,price_vnd
0,929956939,1250000.0
1,835698567,750000.0
2,528898768,550000.0
3,834405505,1000000.0
4,368465111,1150000.0
...,...,...
356441,899658678,5500000.0
356442,913913355,4000000.0
356443,919821994,5500000.0
356444,768866555,10000000.0


In [9]:
def creating_label(df):
    price = np.array(df['price_vnd'])
    label = []

    for i in range(df.shape[0]):
        if price[i] <= 1.6e6:
            label.append(0)
            continue
        if price[i] <= 3.6e6:
            label.append(1)
            continue
        if price[i] <= 6.5e6:
            label.append(2)
            continue
        if price[i] <= 10.8e6:
            label.append(3)
            continue
        if price[i] <= 1.5e7:
            label.append(4)
            continue
        if price[i] <= 6.1e7:
            label.append(5)
            continue
        if price[i] <= 1.5e8:
            label.append(6)
            continue
        if price[i] <= 3.1e8:
            label.append(7)
            continue
        if price[i] <= 5.3e8:
            label.append(8)
            continue

        label.append(9)

    df['label'] = label

    return df


In [10]:
df_ = creating_label(df_)

In [11]:
df_['label'].value_counts()

0    150158
1     80083
2     54418
3     24375
5     18983
4     18304
6      5497
9      2445
7      1603
8       580
Name: label, dtype: int64

In [12]:
df_ = df_.astype({
    'phone_number': 'str'
})

In [13]:
df_ = df_[df_['phone_number'].str.len() == 9]

In [14]:
# df_label_high_2 = df_[df_['label'] > 1]
# df_label_1 = df_[df_['label'] == 1].sample(80000)
# df_label_0 = df_[df_['label'] == 0].sample(150000)

In [15]:
# df_data = pd.concat([df_label_0, df_label_1, df_label_high_2])
# del df_
# df_ = df_data
# df_.reset_index(inplace=True, drop=True)
# del df_data

In [16]:
df_

,phone_number,price_vnd,label
0,929956939,1250000.0,0
1,835698567,750000.0,0
2,528898768,550000.0,0
3,834405505,1000000.0,0
4,368465111,1150000.0,0
...,...,...,...
356441,899658678,5500000.0,2
356442,913913355,4000000.0,2
356443,919821994,5500000.0,2
356444,768866555,10000000.0,3


In [220]:
VIETTEL_HEAD_NUMBER = ['32','33','34','35','37','38','39','86','96','97','98']
MOBI_HEAD_NUMBER = ['70','79','77','76','78','89','90','93']
VINA_HEAD_NUMBER = ['81','84','85','82','83','94','91','88']
VIETNAMMOBI_HEAD_NUMBER = ['56','58','92']
GMOBILE = ['59','99']

dict_list_combo = {
    'LIST_COMBO_6_LOWER_5': ['000000','111111','222222','333333','444444'],
    'LIST_COMBO_5_LOWER_5' : ['00000','11111','22222','33333','44444'],
    'LIST_COMBO_4_LOWER_5 ':  ['0000','1111','2222','3333','4444'],
    'LIST_COMBO_3_LOWER_5' : ['000','111','222','333','444','555','666','777','888','999'],
    'LIST_COMBO_2_LOWER_5' :  ['00','11','22','33','44'],
    'LIST_COMBO_6_HIGHER_5': ['555555','666666','777777','888888','999999'],
    'LIST_COMBO_5_HIGHER_5':  ['55555','66666','77777','88888','99999'],
    'LIST_COMBO_4_HIGHER_5':  ['5555','6666','7777','8888','9999'],
    'LIST_COMBO_3_HIGHER_5':  ['555','666','777','888','999'],
    'LIST_COMBO_2_HIGHER_5': ['55','66','77','88','99'],
    'LIST_LUCKY_NUMBER' : ['68','86','78','79','39','38','83'],
    'LIST_5_INCREASING': ['01234','12345','23456','34567','45678','56789','02468','13579'],
    'LIST_4_INCREASING': ['0123','1234','2345','3456','4567','5678','6789','0246','1357','2468','3579','0369'],
    'LIST_3_INCREASING': ['012','123','234','345','456','567','678','789','024','135','246','357','468','579','036','147','258','369','048','159'],
}

dict_than_tai_combo = {


    'LIST_COMBO_4_THAN_TAI' : ['7997', '8998','7799', '8899','9779', '9889','7979','3939','3579'],
    'LIST_TAXI_NUMBER': ['04','47','07'],
    'LIST_COMBO_4_ONG_DIA': ['6666','3878'],
    'LIST_COMBO_3_ONG_DIA': ['138','238','338','438','538','638','738','838','938'],
    'LIST_COMBO_2_ONG_DIA': ['38','78'],
    'LIST_COMBO_3_THAN_TAI': ['179','579','279','679','979','079','479','879','379'],
    'LIST_COMBO_2_THAN_TAI': ['39','79'],
    'LIST_COMBO_3_LOC_PHAT': ['6688','6868','6886', '8668'],
    'LIST_COMBO_2_LOC_PHAT': ['68','86','88','66'],
    'LIST_COMBO_SO_XAU': ['03','43','53','49','87','47']
}

In [295]:
class FeatureExtraction:
    def __init__(self):

        self.list_dai_cat = ['03','01','00','05','15','16','18','23','24','25','28','30','31','38','40','46','47','66','80','78']
        self.dict_index = {
            '0': False,
            '1': False,
            '2': False,
            '3': False,
            '4': False,
            '5': False,
            '6': False,
            '7': False,
            '8': False
        }

    def preprocessing(self, number):
        _number = number
        if len(number) == 11:
            if number[0:2] == '84':
                _number = number[2:]
        elif number[0] == '0':
            _number = number[1:]
        return _number

    def classification_tele(self, number ):
        head_number = number[0:2]
        if head_number in VIETTEL_HEAD_NUMBER:
            return 'viettel'
        elif head_number in MOBI_HEAD_NUMBER:
            return 'mobi'
        elif head_number in VINA_HEAD_NUMBER:
            return 'vina'
        elif head_number in GMOBILE:
            return 'gmobile'
        elif head_number in VIETNAMMOBI_HEAD_NUMBER:
            return 'vietnammobile'
        else:
            return 'khác'

    def _have_combo(self, number, list_combo, flag):
        if not flag:
            for ele in list_combo:
                if ele in number:
                    _idx = number.find(ele)
                    if _idx > 0:
                        if not self.dict_index[str(_idx)]:
                            for i in range(len(ele)):
                                self.dict_index[str(_idx+i)] = True
                            return ele
        else:
            for ele in list_combo:
                if ele in number:
                    return ele
        return 'no'

    def have_combo(self, number, list_combo, flag=False):
        flag = self._have_combo(number, list_combo, flag)
        if flag != 'no':
            return 'yes'
        return 'no'



    def number_combo(self, number, list_combo, flag=False):
        flag = self._have_combo(number, list_combo, flag)
        if flag != 'no':
            return flag
        return '-1'

    def have_last_combo(self, number, list_combo, number_idx):
        last_n_digits = number[-number_idx:]
        if last_n_digits:
            if last_n_digits in list_combo:
                return 'yes'
            return 'no'
        else:
            return 'no'

    def last_combo(self,number, list_combo, number_idx):
        last_n_digits = number[-number_idx:]
        if last_n_digits:
            if last_n_digits in list_combo:
                return last_n_digits
            return '-1'
        else:
            return '-1'

    def have_repeat_2(self, number):
        for idx in range(0, len(number)-3, 1):
            idx_0 = idx
            idx_1 = idx+1
            idx_2 = idx+2
            idx_3 = idx+3

            str_abc = number[idx_0] + number[idx_1]
            str_abc_2 = number[idx_2] + number[idx_3]
            if str_abc == str_abc_2:
                return 'yes'
        return 'no'



    def repeat_2(self, number):
        for idx in range(0, len(number)-5, 1):
            idx_0 = idx
            idx_1 = idx+1
            idx_2 = idx+2
            idx_3 = idx+3

            str_abc = number[idx_0] + number[idx_1]
            str_abc_2 = number[idx_2] + number[idx_3]
            if str_abc == str_abc_2:
                return str_abc
        return '-1'

    def have_repeat_3(self, number):
        for idx in range(0, len(number)-5, 1):
            idx_0 = idx
            idx_1 = idx+1
            idx_2 = idx+2
            idx_3 = idx+3
            idx_4 = idx+4
            idx_5 = idx+5
            str_abc = number[idx_0] + number[idx_1] + number[idx_2]
            str_abc_2 = number[idx_3] + number[idx_4] + number[idx_5]
            if str_abc == str_abc_2:
                return 'yes'
        return 'no'

    def repeat_3(self, number):
        for idx in range(0, len(number)-5, 1):
            idx_0 = idx
            idx_1 = idx+1
            idx_2 = idx+2
            idx_3 = idx+3
            idx_4 = idx+4
            idx_5 = idx+5
            str_abc = number[idx_0] + number[idx_1] + number[idx_2]
            str_abc_2 = number[idx_3] + number[idx_4] + number[idx_5]
            if str_abc == str_abc_2:
                return str_abc
        return '-1'

    def have_repeat_ababab(self, number):
        for idx in range(0, len(number)-5, 1):
            idx_0 = idx
            idx_1 = idx+1
            idx_2 = idx+2
            idx_3 = idx+3
            idx_4 = idx+4
            idx_5 = idx+5
            str_ab = number[idx_0] + number[idx_1]
            str_ab_2 = number[idx_2] + number[idx_3]
            str_ab_3 = number[idx_4] + number[idx_5]
            if str_ab == str_ab_2 and str_ab == str_ab_3:
                return 'yes'
        return 'no'
    def repeat_ababab(self, number):
        for idx in range(0, len(number)-5, 1):
            idx_0 = idx
            idx_1 = idx+1
            idx_2 = idx+2
            idx_3 = idx+3
            idx_4 = idx+4
            idx_5 = idx+5
            str_ab = number[idx_0] + number[idx_1]
            str_ab_2 = number[idx_2] + number[idx_3]
            str_ab_3 = number[idx_4] + number[idx_5]
            if str_ab == str_ab_2 and str_ab == str_ab_3:
                return str_ab + str_ab_2 + str_ab_2
        return '-1'

    def have_repeat_abcdabcd(self, number):
        for idx in range(0, len(number)-7, 1):
            idx_0 = idx
            idx_1 = idx+1
            idx_2 = idx+2
            idx_3 = idx+3
            idx_4 = idx+4
            idx_5 = idx+5
            idx_6 = idx+6
            idx_7 = idx+7
            str_ab = number[idx_0] + number[idx_1] + number[idx_2] + number[idx_3]
            str_ab_2 = number[idx_4] + number[idx_5] + number[idx_6] + number[idx_7]
            if str_ab == str_ab_2:
                return 'yes'
        return 'no'

    def repeat_abcdabcd(self, number):
        for idx in range(0, len(number)-7, 1):
            idx_0 = idx
            idx_1 = idx+1
            idx_2 = idx+2
            idx_3 = idx+3
            idx_4 = idx+4
            idx_5 = idx+5
            idx_6 = idx+6
            idx_7 = idx+7
            str_ab = number[idx_0] + number[idx_1] + number[idx_2] + number[idx_3]
            str_ab_2 = number[idx_4] + number[idx_5] + number[idx_6] + number[idx_7]
            if str_ab == str_ab_2:
                return str_ab
        return '-1'
    def have_dual_sim_3(self, number):
        for idx in range(0, len(number)-5, 1):
            if (number[idx] == number[idx+1]) and (number[idx+2] == number[idx+3]) and (number[idx+4] == number[idx+5]):
                return 'yes'
        return 'no'

    def dual_sim_3(self, number):
        for idx in range(0, len(number)-5, 1):
            if (number[idx] == number[idx+1]) and (number[idx+2] == number[idx+3]) and (number[idx+4] == number[idx+5]):
                return number[idx:idx+5]
        return '-1'

    def have_double_combo(self, number, list_combo):
        count = 0
        for combo in list_combo:
            if combo in number:
                count += 1
        if count == 2:
            return 'yes'
        else:
            return 'no'

    def double_combo(self, number, list_combo):
        count = 0
        double_combo = ''
        for combo in list_combo:
            if combo in number:
                count += 1
                double_combo += combo
        if count == 2:
            return double_combo
        else:
            return '-1'

    def have_double_number(self, number):
        for idx in range(0, len(number)-1,1):
            str_numbers = number[idx] + number[idx+1]
            for _idx in range(idx+2, len(number) -1, 1):
                str_numbers_ss = number[_idx] + number[_idx+1]
                if str_numbers == str_numbers_ss:
                    return 'yes'
        return 'no'

    def double_number(self, number):
        for idx in range(0, len(number)-1,1):
            str_numbers = number[idx] + number[idx+1]
            for _idx in range(idx+2, len(number) -1, 1):
                str_numbers_ss = number[_idx] + number[_idx+1]
                if str_numbers == str_numbers_ss:
                    return str_numbers
        return '-1'

    def have_sim_ganh_dao(self, number):
        idx = 1
        type_ganh_dao = 0
        flag = False
        while not flag and idx < len(number) - 1:
            if number[idx-1] == number[idx+1]:
                type_ganh_dao = 1

                if (idx > 2 and idx < len(number) - 2) and number[idx-2] == number[idx+2]:
                    type_ganh_dao = 2
                    flag = True
            elif (idx > 2 and idx < len(number) - 2) and number[idx-1] == number[idx+2] and number[idx-2] == number[idx+1]:
                type_ganh_dao = 3
                flag = True
            idx += 1
        if type_ganh_dao == 0:
            return 'khác'
        elif type_ganh_dao == 1:
            return 'ganh_dao_don'
        elif type_ganh_dao == 2:
            return 'ganh_dao_kep'
        elif type_ganh_dao == 3:
            return 'ganh_dao_kep_tien'

    def sim_ganh_dao(self, number):
        idx = 1
        type_ganh_dao = 0
        string_ganh_dao = 'khác'
        flag = False
        while not flag and idx < len(number) - 1:
            if number[idx-1] == number[idx+1]:
                string_ganh_dao = number[idx-1:idx+2]
                if (idx > 2 and idx < len(number) - 2) and number[idx-2] == number[idx+2]:
                    string_ganh_dao = number[idx-2:idx+3]
                    flag = True
            elif (idx > 2 and idx < len(number) - 2) and number[idx-1] == number[idx+2] and number[idx-2] == number[idx+1]:
                string_ganh_dao = number[idx-2:idx+3]
            idx += 1

        return string_ganh_dao

    def _dai_cat(self, number):
        quotient_dai_cat = int(number[-3:]) / 80
        decimal_dai_cat = quotient_dai_cat - int(quotient_dai_cat)
        int_dai_cat = decimal_dai_cat * 80
        if int_dai_cat in self.list_dai_cat:
            return True
        return False

    def have_dai_cat(self, number):
        if self._dai_cat(number):
            return 'yes'
        return 'no'

    def have_year_born(self, number):
        last_4_th = number[-4:]
        if 1940 <= int(last_4_th) <= 2022:
            return 'yes'
        return 'no'

    def year_born(self, number):
        last_4_th = number[-4:]
        if 1940 <= int(last_4_th) <= 2022:
            return last_4_th
        return '-1'

    def have_sim_dao(self, number):
        for idx in range(1, len(number) - 2, 1):
            if number[idx] == number[idx+1] and number[idx-1] == number[idx+2]:
                return 'yes'
        return 'no'

    def sim_dao(self, number):
        str_sim_dao = '-1'
        for idx in range(1, len(number) - 2, 1):
            if number[idx] == number[idx+1] and number[idx-1] == number[idx+2]:
                str_sim_dao = number[idx-1:idx+3]
        return str_sim_dao

    def feature_extract(self, _number):
        number = str(_number)
        dict_feature = {}
        number = self.preprocessing(number)

        dict_feature['tele'] = self.classification_tele(number)
        dict_feature['heads'] = number[0:2]
        dict_feature['last_2'] = number[-2:]
        dict_feature['last_3'] = number[-3:]
        feature_combo = '2'
        for index in range(len(number)):

            dict_feature['index_{}'.format(index)] = number[index]


        _set_list = set(number[1:])
        dict_feature['count_number'] = len(_set_list)

        for key, value in dict_list_combo.items():
            len_number = len(value[0])
            if 'LIST' in key:
                feature_name = str.lower(key[5:])
            else:
                feature_name = str.lower(key)
            dict_feature['have_last_{}'.format(feature_name)] = self.have_last_combo(number, value, len_number)
            dict_feature['last_{}'.format(feature_name)] = self.last_combo(number,value, len_number)
            if dict_feature['have_last_{}'.format(feature_name)] == 'no':
                dict_feature['have_{}'.format(feature_name)] = self.have_combo(number, value, flag=True)
                dict_feature['{}'.format(feature_name)] = self.number_combo(number, value, flag=True)
            else:
                dict_feature['have_{}'.format(feature_name)] = 'no'
                dict_feature['{}'.format(feature_name)] = '-1'
        flag_than_tai = 0
        for k, v in dict_than_tai_combo.items():
            len_number = len(v[0])
            if 'LIST' in k:
                feature_name = str.lower(k[5:])
            else:
                feature_name = str.lower(k)
            if flag_than_tai == 0:
                dict_feature['have_last_{}'.format(feature_name)] = self.have_last_combo(number, v, len_number)
                dict_feature['last_{}'.format(feature_name)] = self.last_combo(number,v, len_number)
                if dict_feature['have_last_{}'.format(feature_name)] != 'no':
                    flag_than_tai = 1
            else:
                dict_feature['have_last_{}'.format(feature_name)] = 'no'
                dict_feature['last_{}'.format(feature_name)] = '-1'



        dict_feature['have_repeat_2'] = self.have_repeat_2(number)
        dict_feature['repeat_2'] = self.repeat_2(number)
        dict_feature['have_repeat_3'] = self.have_repeat_3(number)
        dict_feature['repeat_3'] = self.repeat_3(number)



        dict_feature['have_repeat_ababab'] = self.have_repeat_ababab(number)
        dict_feature['repeat_ababab'] = self.repeat_ababab(number)



        dict_feature['have_repeat_abcdabcd'] = self.have_repeat_abcdabcd(number)
        dict_feature['repeat_abcdabcd'] = self.repeat_abcdabcd(number)
        dict_feature['have_dual_sim_3'] = self.have_dual_sim_3(number)
        dict_feature['dual_sim_3'] = self.dual_sim_3(number)

        dict_feature['have_double_combo'] = self.have_double_combo(number, dict_list_combo['LIST_COMBO_2_LOWER_5'])
        dict_feature['double_combo'] = self.double_combo(number, dict_list_combo['LIST_COMBO_2_LOWER_5'])

        dict_feature['have_double_combo_higher_5'] = self.have_double_combo(number, dict_list_combo['LIST_COMBO_2_HIGHER_5'])
        dict_feature['double_combo_higher_5'] = self.double_combo(number, dict_list_combo['LIST_COMBO_2_HIGHER_5'])

        dict_feature['have_double_combo_3'] = self.have_double_combo(number, dict_list_combo['LIST_COMBO_3_LOWER_5'])
        dict_feature['double_combo_3'] = self.double_combo(number, dict_list_combo['LIST_COMBO_3_LOWER_5'])

        dict_feature['have_double_combo_3_higher_5'] = self.have_double_combo(number, dict_list_combo['LIST_COMBO_3_HIGHER_5'])
        dict_feature['double_combo_3_higher_5'] = self.double_combo(number, dict_list_combo['LIST_COMBO_3_HIGHER_5'])

        dict_feature['have_double_combo_3_taxi'] = self.have_double_combo(number, dict_than_tai_combo['LIST_TAXI_NUMBER'])
        dict_feature['double_combo_3_taxi'] = self.double_combo(number, dict_than_tai_combo['LIST_TAXI_NUMBER'])

        dict_feature['have_double_phat_loc'] =self.have_double_combo(number, dict_than_tai_combo['LIST_COMBO_2_LOC_PHAT'])
        dict_feature['double_phat_loc'] = self.double_combo(number, dict_than_tai_combo['LIST_COMBO_2_LOC_PHAT'])

        dict_feature['have_double_number'] = self.have_double_number(number)
        dict_feature['double_number'] = self.double_number(number)
        dict_feature['have_sim_ganh_dao'] = self.have_sim_ganh_dao(number[1:])
        dict_feature['sim_ganh_dao'] = self.sim_ganh_dao(number[1:])
        dict_feature['have_year_born'] = self.have_year_born(number)
        dict_feature['year_born'] = self.year_born(number)
        dict_feature['have_sim_dao'] = self.have_sim_dao(number[1:])
        dict_feature['sim_dao'] = self.sim_dao(number[1:])
        dict_feature['dai_cat'] = self.have_dai_cat(number)
        return dict_feature


ImportError: cannot import name '_ClassNamePrefixFeaturesOutMixin' from 'sklearn.base' (C:\Users\thinhnd\miniconda3\envs\py3.8\lib\site-packages\sklearn\base.py)

In [296]:
_t = Timer()
_t.start('start feature extraction')
list_dict = []
feature_extract = FeatureExtraction()
for idx, row in df_.iterrows():
    row_feature = feature_extract.feature_extract(row['phone_number'])
    row_feature['phone_number'] = row['phone_number']
    row_feature['price'] = row['price_vnd']
    row_feature['label'] = row['label']
    list_dict.append(row_feature)
_t.stop()

[INFO] start feature extraction finished in 69.530 s.


In [297]:
_df = pd.DataFrame(list_dict)

In [298]:
columns = _df.columns

In [299]:
columns = [column for column in columns if column != 'phone_number' and column != 'price' and column != 'label']

In [300]:
from sklearn import preprocessing
encoder = {}
for col in columns:
    encoder[col] = preprocessing.LabelEncoder()
    _df[col] = encoder[col].fit_transform(_df[col])

In [301]:
_df

,tele,heads,last_2,last_3,index_0,index_1,index_2,index_3,index_4,index_5,...,have_sim_ganh_dao,sim_ganh_dao,have_year_born,year_born,have_sim_dao,sim_dao,dai_cat,phone_number,price,label
0,3,30,39,939,4,2,9,9,5,6,...,0,1778,0,0,0,0,0,929956939,1250000.0,0
1,5,21,67,567,3,3,5,6,9,8,...,3,1891,0,0,0,0,0,835698567,750000.0,0
2,1,8,68,768,1,2,8,8,9,8,...,0,1697,0,0,0,0,0,528898768,550000.0,0
3,5,21,5,505,3,3,4,4,0,5,...,2,947,0,0,1,6,0,834405505,1000000.0,0
4,1,4,11,111,0,6,8,4,6,5,...,0,203,0,0,0,0,0,368465111,1150000.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
356441,2,27,78,678,3,9,9,6,5,8,...,3,1891,0,0,0,0,0,899658678,5500000.0,2
356442,5,29,55,355,4,1,3,9,1,3,...,3,1891,0,0,0,0,0,913913355,4000000.0,2
356443,5,29,94,994,4,1,9,8,2,1,...,3,1891,1,55,0,0,0,919821994,5500000.0,2
356444,2,15,55,555,2,6,8,8,6,6,...,0,1042,0,0,1,69,0,768866555,10000000.0,3


In [302]:
X, y = _df, _df['label']

In [303]:
X_train, X_test, y_train, y_test = train_test_split(X[columns], y, test_size=0.15 ,random_state=14)

In [ ]:
from imblearn.over_sampling import SMOTE
oversample = SMOTE()

In [ ]:
X_train, y_train = oversample.fit_resample(X_train, y_train)

In [ ]:
import xgboost as xgb

In [ ]:
reg =  xgb.XGBClassifier(random_state=0)
reg.fit(X_train, y_train)

In [ ]:
importances = reg.feature_importances_
indices = np.argsort(importances)

In [ ]:
feature_names = np.array(columns)

In [ ]:
data={'feature_names':feature_names,'feature_importance':importances}
fi_df = pd.DataFrame(data)

#Sort the DataFrame in order decreasing feature importance
fi_df.sort_values(by=['feature_importance'], ascending=False,inplace=True)

#Define size of bar plot
plt.figure(figsize=(10,8))
#Plot Searborn bar chart
sns.barplot(x=fi_df['feature_importance'][:20], y=fi_df['feature_names'][:20])
#Add chart labels
plt.xlabel('FEATURE IMPORTANCE')
plt.ylabel('FEATURE NAMES')


In [ ]:
y_pred = reg.predict(X_test)

In [ ]:
print(classification_report(np.array(y_test), y_pred))

In [ ]:
cm =  confusion_matrix(y_test, y_pred)

In [ ]:
cm


In [ ]:
list_idx = []
label_true = 1
label_pred = 0
for idx in range(len(y_pred)):
    if y_pred[idx] == label_pred and y_test.iloc[idx] == label_true:
        list_idx.append(idx)
X_test_cm = pd.DataFrame()
for col in columns:
    X_test_cm[col] = encoder[col].inverse_transform(X_test[col])

test_X = X.loc[X_test.iloc[list_idx].index]
test_X['predict'] = label_pred
for col in columns:
    test_X[col] = encoder[col].inverse_transform(test_X[col])
test_X

In [323]:
df_

,phone_number,price_vnd,label
0,929956939,1250000.0,0
1,835698567,750000.0,0
2,528898768,550000.0,0
3,834405505,1000000.0,0
4,368465111,1150000.0,0
...,...,...,...
356441,899658678,5500000.0,2
356442,913913355,4000000.0,2
356443,919821994,5500000.0,2
356444,768866555,10000000.0,3


In [73]:
test_X['price'] = test_X['price']/1000

In [74]:
test_X

,tele,index_0,index_1,index_2,index_3,index_4,index_5,index_6,index_7,index_8,...,sim_ganh_dao,have_year_born,year_born,have_sim_dao,sim_dao,dai_cat,phone_number,price,label,predict
279976,viettel,8,6,7,8,9,6,6,6,6,...,666,no,-1,yes,6666,no,867896666,444000.000,8,6
253416,vina,9,1,7,7,1,1,1,1,1,...,11111,no,-1,yes,1111,no,917711111,398000.000,8,6
256620,vina,8,2,6,7,8,9,9,9,9,...,999,no,-1,yes,9999,no,826789999,390000.000,8,6
335875,viettel,9,7,7,1,2,1,2,1,2,...,12121,no,-1,no,-1,no,977121212,399000.000,8,6
328880,mobi,7,8,8,9,9,8,8,8,8,...,888,no,-1,yes,8888,no,788998888,320000.000,8,6
305105,viettel,9,7,9,3,3,9,3,3,9,...,33933,no,-1,yes,9339,no,979339339,450000.000,8,6
234225,mobi,9,0,5,1,2,1,2,1,2,...,12121,no,-1,no,-1,no,905121212,450000.000,8,6
348721,viettel,9,6,5,8,6,8,8,6,8,...,868,no,-1,yes,6886,no,965868868,350000.000,8,6
320602,viettel,9,7,3,8,9,6,6,6,6,...,666,no,-1,yes,6666,no,973896666,339000.000,8,6
299729,vina,9,1,6,6,5,6,5,6,5,...,65656,no,-1,no,-1,no,916656565,313000.000,8,6


In [267]:
df_['price_vnd'].loc[test_X.index] = test_X['price']

In [268]:
df_['price_vnd'][test_X.index]

Series([], Name: price_vnd, dtype: float64)

In [288]:
df_.to_csv('../data/train_data_1.csv',index=False)